# Inventory Management Environments

> To be written.

In [ ]:
#| default_exp envs.base

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import gymnasium as gym
from abc import ABC, abstractmethod

from ddopnew.utils import MDPInfo

In [ ]:
#| export
class BaseEnvironment(gym.Env, ABC):

    def __init__(self, mdp_info: MDPInfo) -> None:
        """
        Constructor.

        Args:
            mdp_info (MDPInfo): the information of the environment.

        """
        super().__init__()

        self.index = 0
        self.mdp_info = mdp_info

    @property
    def info(self):
        """
        Returns:
             An object containing the info of the environment.

        """
        return self._mdp_info

    @abstractmethod
    def set_action_space(self):
        """
        Set the action space of the environment.

        """
        pass

    @abstractmethod
    def set_observation_space(self):
        """
        Set the observation space of the environment.

        """
        pass

    @abstractmethod
    def get_observation(self):
        """
        Return the current observation.

        """
        pass
    
    def handle_index(self):
        """
        Handle the index of the environment.

        """

        self.index += 1
        truncated = True if self.index >= self.mdp_info.horizon else False
        
        return truncated

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()